# GPU Programming with CUDA: Matrix Product

Execute the cells below to run the code in google colab environment.

In [1]:
! pip install git+git://github.com/frehseg/nvcc4jupyter.git

  Cloning git://github.com/frehseg/nvcc4jupyter.git to /tmp/pip-req-build-2ppbud7b
  Running command git clone -q git://github.com/frehseg/nvcc4jupyter.git /tmp/pip-req-build-2ppbud7b
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.1-cp36-none-any.whl size=2095 sha256=24ae0ed310e98cbe3d5902178bc9961e7e7addf65308b31140661f8ac0e13ee7
  Stored in directory: /tmp/pip-ephem-wheel-cache-tlhii53z/wheels/a4/a5/24/17a2b61f9a725a10155cc6fca753aae28436921df21fa16114
Successfully built NVCCPlugin


In [0]:
%load_ext nvcc_plugin

## Classic product version without shared memory  
  
In this naive version, we compute the product $C = A * B$ in global memory with the standard matrix multiplication algorithm.

* Each thread calculates an element of C
* Each thread accesses to a whole line of A and a whole column of B

**Downsides of this method**:
* The access to data is non-aligned and scattered
* Pb of coalescing
* Repeated data access  
  
## Shared memory version

Optimization: we can divide the matrix in different tiles and compute the product tile after tile.

With this, we have:

* Cooperative loading of a data tile in shared memory with regular accesses to global memory + quick accesses once in shared memory.
* Some synchronization points to ensure that the data of partial results computed by threads are correctly loaded.

In [18]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <time.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>

#define TILE_WIDTH 32
#define SIZE 20


/********************** CPU Version **************************/
void CPUMatMul(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_RowA, int nb_RowB)
{
  for (int i = 0; i < nb_RowA; i++) {
        for (int j = 0; j < nb_ColB; j++) {
            int sum = 0;
            for (int k = 0; k < nb_ColA; k++)
                sum = sum + A[i * nb_ColA + k] * B[k * nb_ColB + j];
            C[i * nb_ColB + j] = sum;
        }
    }
}

/********************** Classic Product Version **************************/
__global__
void classicMatMul(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_RowA, int nb_RowB)
{
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;
  if (row < nb_RowA && col < nb_RowB) {
      for(int i = 0; i < nb_ColA; i++){
          C[row * nb_ColB + col] += A[row * nb_ColA + i] * B[i * nb_ColB + col];
      }
  }
}

/********************** Shared Memory Version ****************************/
__global__
void sharedMemMatMul(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_RowA, int nb_RowB)
{
    int blockRow = blockIdx.y;
    int blockCol = blockIdx.x;

    // Matrix C Tile
    float* Csub = &C[nb_ColB * TILE_WIDTH * blockRow + TILE_WIDTH * blockCol];

    // value of C computed by one thread
    float Cvalue = 0;

    // Matrix C Tile Row & col
    int row = threadIdx.y;
    int col = threadIdx.x;

    // We go through each tile of A and B that are
    // required to compute the C tile
    for (int m = 0; m < (nb_ColA / TILE_WIDTH); ++m) {

        // A Tile
        float* Asub = &A[nb_ColA * TILE_WIDTH * blockRow + TILE_WIDTH * m];

        // B Tile
        float* Bsub = &B[nb_ColB * TILE_WIDTH * m + TILE_WIDTH * blockCol];

        // Shared memory for A & B tiles
        __shared__ float As[TILE_WIDTH][TILE_WIDTH];
        __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];

        // Load A & B values in shared memory with a thread
        As[row][col] = Asub[row * nb_ColA + col];
        Bs[row][col] = Bsub[row * nb_ColB + col];

        // Synchronization before computing C
        __syncthreads();
        // Multiply Asub and Bsub together
        for (int e = 0; e < TILE_WIDTH; ++e)
            Cvalue += As[row][e] * Bs[e][col];

        // Synch threads before other A & B tiles
        __syncthreads();
    }

    // Write in C tile the cumulated value
    Csub[row * nb_ColA + col] = Cvalue;
}

/********************** main **************************/
int main(void)
{
  float *A, *B, *C, *gpu_A, *gpu_B, *gpu_C;
  int nbRowA, nbRowB, nbColA, nbColB;

  
  nbRowA = TILE_WIDTH * SIZE;
  nbRowB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;

  A = (float*) malloc(nbRowA * nbColA * sizeof(float));
  B = (float*) malloc(nbRowB * nbColB * sizeof(float));
  C = (float*) malloc(nbRowA * nbColB * sizeof(float));

  /* GPU space allocation */
  cudaMalloc((void**) &gpu_A, nbRowA * nbColA * sizeof(float));
  cudaMalloc((void**) &gpu_B, nbRowB * nbColB * sizeof(float));
  cudaMalloc((void**) &gpu_C, nbRowA * nbColB * sizeof(float));

  /* A & B initialization */
  for (int i = 0; i < nbRowA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbRowB * nbColB; i++) {
    B[i] = 2.0;
  }

  /* Copy of A & B on GPU */
  cudaMemcpy(gpu_A, A, nbRowA * nbColA * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(gpu_B, B, nbRowB * nbColB * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(gpu_C, C, nbRowA * nbColB * sizeof(float), cudaMemcpyHostToDevice);


 dim3 threadsPerBlock (TILE_WIDTH, TILE_WIDTH);
 dim3 blocksPerGrid (nbRowA / threadsPerBlock.y, nbRowB / threadsPerBlock.x);

 double cpu_time = 0.0;
 float cpu_max_error = 0.0f;
 double classic_time = 0.0;
 float classic_max_error = 0.0f;
 double shared_mem_time = 0.0;
 float shared_mem_max_error = 0.0f;

 int n_iter = 10;
 for(int i = 0; i < n_iter; i++){
     
 /************ CPU **********************************/
 clock_t start = clock();
 CPUMatMul(A, B, C, nbColA, nbColB, nbRowA, nbRowB);
 clock_t end = clock();
 cpu_time += ((double) end-start)/CLOCKS_PER_SEC; // in seconds
 if(i == 0){
    /* Vérification du résultat*/
    for (int i = 0; i < nbRowA * nbColB; i++){
        cpu_max_error = max(cpu_max_error, abs(C[i]- 2*nbRowB));
    }
  }

 /************ Classic Product Version **************/
 start = clock();
 classicMatMul<<<blocksPerGrid, threadsPerBlock>>>(gpu_A, gpu_B, gpu_C, nbColA, nbColB, nbRowA, nbRowB);
 end = clock();
 classic_time += ((double) end-start)/CLOCKS_PER_SEC; // in seconds
 if(i == 0){
    cudaMemcpy(C, gpu_C, nbRowA * nbColB * sizeof(float), cudaMemcpyDeviceToHost);
    /* Vérification du résultat*/
    for (int i = 0; i < nbRowA * nbColB; i++){
        classic_max_error = max(classic_max_error, abs(C[i]- 2*nbRowB));
    }
  }

 /************ Shared Memory Version ****************/
 start = clock();
 sharedMemMatMul<<<blocksPerGrid, threadsPerBlock>>>(gpu_A, gpu_B, gpu_C, nbColA, nbColB, nbRowA, nbRowB);
 end = clock();
 shared_mem_time += ((double) end-start)/CLOCKS_PER_SEC; // in seconds
 if(i == 0){
    cudaMemcpy(C, gpu_C, nbRowA * nbColB * sizeof(float), cudaMemcpyDeviceToHost);
    /* Vérification du résultat*/
    for (int i = 0; i < nbRowA * nbColB; i++){
        shared_mem_max_error = max(shared_mem_max_error, abs(C[i]- 2*nbRowB));
    }
  }
 }
 
 printf("\n************** CPU ************************");
 printf("\nCPU version took %f seconds to execute \n", cpu_time/n_iter);
 printf("Max error: %f\n", cpu_max_error);
 
 printf("\n************** Classic Product ************");
 printf("\nClassic version took %f seconds to execute \n", classic_time/n_iter);
 printf("Max error: %f\n", classic_max_error);

 printf("\n************** Shared Memory **************");
 printf("\nShared memory version took %f seconds to execute \n", shared_mem_time/n_iter);
 printf("Max error: %f\n", shared_mem_max_error);

  /* Libération de la mémoire sur le GPU*/ 
  cudaFree(gpu_A);
  cudaFree(gpu_B);
  cudaFree(gpu_C);

  free(A);
  free(B);
  free(C);
 
 printf("%s\n", cudaGetErrorString(cudaGetLastError()));
}



************** CPU ************************
CPU version took 1.861266 seconds to execute 
Max error: 0.000000

************** Classic Product ************
Classic version took 0.000045 seconds to execute 
Max error: 0.000000

************** Shared Memory **************
Shared memory version took 0.000008 seconds to execute 
Max error: 0.000000
no error



In [14]:
%%cu
#include <stdlib.h>
#include <stdio.h>

#include <time.h>

#include <cuda.h>
#include <cuda_runtime.h>
#include <helper_cuda.h>

#define TILE_WIDTH 32
#define SIZE 300
/********************** kernel **************************/
__global__
void matmul(float *A, float *B, float *C, int nb_ColA, int nb_ColB, int nb_RowA, int nb_RowB)
{
    int blockRow = blockIdx.y;
    int blockCol = blockIdx.x;

    // Tuile de la matrice C
    float* Csub = &C[nb_ColB * TILE_WIDTH * blockRow + TILE_WIDTH * blockCol];

    // Valeur de C calculée par un thread
    float Cvalue = 0;

    // Row et colonne dans une tuile de C
    int row = threadIdx.y;
    int col = threadIdx.x;

    // On parcoure les differentes tuiles de A et B
    // qui sont necessaires au calcul de la tuile C
    for (int m = 0; m < (nb_ColA / TILE_WIDTH); ++m) {

        // Tuile de A
        float* Asub = &A[nb_ColA * TILE_WIDTH * blockRow + TILE_WIDTH * m];

        // Tuile de B
        float* Bsub = &B[nb_ColB * TILE_WIDTH * m + TILE_WIDTH * blockCol];

        // Memoire partagee pour stocker les tuiles de A et B
        __shared__ float As[TILE_WIDTH][TILE_WIDTH];
        __shared__ float Bs[TILE_WIDTH][TILE_WIDTH];

        // Chargement de dans la memoire partagee
        // des valeurs de A et B par un thread
        As[row][col] = Asub[row * nb_ColA + col];
        Bs[row][col] = Bsub[row * nb_ColB + col];

        // Synchronisation des threads avant calcul pour C
        __syncthreads();
        // Multiply Asub and Bsub together
        for (int e = 0; e < TILE_WIDTH; ++e)
            Cvalue += As[row][e] * Bs[e][col];

        // Autre Synchronisation avant de charger des nouvelles
        // tuiles pour A et B
        __syncthreads();
    }

    // Ecriture dans C par le thread de la valeur accumulee
    Csub[row * nb_ColA + col] = Cvalue;
}

/********************** main **************************/
int main(void)
{
  float *A, *B, *C, *gpu_A, *gpu_B, *gpu_C;
  int nbRowA, nbRowB, nbColA, nbColB;

  
  nbRowA = TILE_WIDTH * SIZE;
  nbRowB = TILE_WIDTH * SIZE;
  nbColA = TILE_WIDTH * SIZE;
  nbColB = TILE_WIDTH * SIZE;

  A = (float*) malloc(nbRowA * nbColA * sizeof(float));
  B = (float*) malloc(nbRowB * nbColB * sizeof(float));
  C = (float*) malloc(nbRowA * nbColB * sizeof(float));

  /*Allocation de l'espace pour le GPU */
  cudaMalloc((void**) &gpu_A, nbRowA * nbColA * sizeof(float));
  cudaMalloc((void**) &gpu_B, nbRowB * nbColB * sizeof(float));
  cudaMalloc((void**) &gpu_C, nbRowA * nbColB * sizeof(float));

  /* Initialisation de A et B*/
  for (int i = 0; i < nbRowA * nbColA; i++) {
    A[i] = 1.0;
  }

  for (int i = 0; i < nbRowB * nbColB; i++) {
    B[i] = 2.0;
  }

  /* Copie de A et B sur le GPU */
  cudaMemcpy(gpu_A, A, nbRowA * nbColA * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(gpu_B, B, nbRowB * nbColB * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(gpu_C, C, nbRowA * nbColB * sizeof(float), cudaMemcpyHostToDevice);


  /* Lancement du kernel avec mesure du temps */
 dim3 threadsPerBlock (TILE_WIDTH, TILE_WIDTH);
 dim3 blocksPerGrid (nbRowA / threadsPerBlock.y, nbRowB / threadsPerBlock.x);

 clock_t start = clock();

 matmul<<<blocksPerGrid, threadsPerBlock>>>(gpu_A, gpu_B, gpu_C, nbColA, nbColB, nbRowA, nbRowB);

 clock_t end = clock();
 double time_taken = ((double) end-start)/CLOCKS_PER_SEC; // in seconds
 printf("\n Kernel took %f seconds to execute\n\n", time_taken);

  cudaMemcpy(C, gpu_C, nbRowA * nbColB * sizeof(float), cudaMemcpyDeviceToHost);

  /* Vérification du résultat*/
  float maxError = 0.0f;
  for (int i = 0; i < nbRowA * nbColB; i++){
      maxError = max(maxError, abs(C[i]- 2*nbRowB));
  }
  printf("Max error: %f\n", maxError);

  /* Libération de la mémoire sur le GPU*/ 
  cudaFree(gpu_A);
  cudaFree(gpu_B);
  cudaFree(gpu_C);

  free(A);
  free(B);
  free(C);
 
 printf("%s\n", cudaGetErrorString(cudaGetLastError()));
}


 Kernel took 0.000028 seconds to execute

Max error: 0.000000
no error

